In [ ]:
# RNN
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random

def rnn_path_to_vector(path):
    output = []
    path_bytes = [i for i in bytes(path, 'ascii')]
    for b in path_bytes:
        entry = np.zeros(95)
        entry[b - 0x20] = 1
        output.append(entry)
    return np.array(output)

X_train, X_test, y_train, y_test = train_test_split([rnn_path_to_vector(x) for x in X], y, test_size=0.33, random_state=1234)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

rnn_model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(1, input_shape=[None, 95], activation="relu"),
])

rnn_model.compile(
    optimizer='adam', 
    loss="binary_crossentropy", 
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)

def train_generator():
    for i in range(len(X_train)):
        x = X_train[i]
        y = y_train[i]
        yield np.array([x]), np.array([y])

def eval_generator():
    for i in range(len(X_test)):
        x = X_test[i]
        y = y_test[i]
        yield np.array([x]), np.array([y])

rnn_model.fit(train_generator())
print("evaluate: ", rnn_model.evaluate(eval_generator()))

def devectorize(vector):
    out = []
    for i in vector:
        for j in range(len(i)):
            if i[j] == 1: out.append(chr(0x20+j))
    return ''.join(out)

TP = []
FP = []
TN = []
FN = []

for i in range(len(X_train)):
    out = rnn_model.predict(np.asarray([X_train[i]]), verbose=0)[0][0]
    strin = devectorize(X_train[i])
    if out > 0.5:
        if y_train[i] == 1: TP.append(strin)
        else: FP.append(strin)
    else:
        if y_train[i] == 1: FN.append(strin)
        else: TN.append(strin)

print(len(TP))
print(len(FP))
print(len(TN))
print(len(FN))

print(len(y_train))
for i in range(len(y_train)):
    if y_train[i] == 1:
        print(devectorize(X_train[i]))
        out = rnn_model(np.asarray([X_train[i]]), training=False)[0][0]
        print(out.numpy())
        print(y_train[i])